## 该notebook用于替换某一个特定晶面上的原子

In [1]:
from pyyabc.io.vasp import read_vasp, write_vasp
from pyyabc.crystal.structure import symbol2number as s2n
import numpy
import os, shutil
from pyyabc.toolkit.derivetool import remove_redundant
from pyyabc.crystal.derive import PermutationGroup
from pyyabc.crystal.structure import Cell

In [2]:
symprec = 1e-3
comprec = 1e-3

substitutes = 'Fe'

comment = 'CuxFe'
folder = 'CuFe_101-plane'

# hkl-index
plane_index = [1, 0, 1]

# In[4]:

cell = read_vasp("./Cu_fcc.vasp")

#### 创建一个文件夹

In [3]:
if not os.path.exists('./'+folder):
    os.mkdir('./'+folder)
else:
    shutil.rmtree('./'+folder)
    os.mkdir('./'+folder)

### 主函数

In [4]:
def main():
    h = numpy.array([[2,0,0],[0,2,0],[0,0,2]])
    hfpg = PermutationGroup(cell, h)
    perms = hfpg.get_symmetry_perms(symprec)
    supercell = cell.extend(h)
    pos = supercell.positions
    _sites = []
    _tmp = 0
    for ii in range(len(supercell.atoms)):
        if numpy.dot(pos[ii], numpy.array(plane_index)) == 1:
            _sites.append((supercell.atoms[ii], s2n(substitutes)))
            _tmp += 1
        else:
            _sites.append((supercell.atoms[ii],))
        e_num = [_tmp - 4, 4]
    for mol, d in remove_redundant(supercell.positions, _sites, perms, e_num):
        c = Cell(supercell.lattice, mol[0], mol[1])
        yield (c, d)

### 写入POSCRA

In [5]:
idx = 0
for c, _ in main():
    filepath = os.path.join('./'+folder, '{:s}_id{:d}'.format(comment, idx))
    write_vasp(c, filepath)       
    idx += 1
